### Reindexing

`reindex` 是一个较为常见的 pandas 方法，它将基于符合新索引的数据创建一个新对象：

In [1]:
import pandas as pd
import numpy as np

In [2]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index = ['d', 'b', 'a', 'c'])

In [3]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [4]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

In [5]:
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

可以看到根据新的索引，之前的 Series 已经被重新排列；任何不存在对象的索引将会被赋值 NaN

对于时间序列数据，我们可以通过`method`来对索引进行自动补充：

In [3]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index = [0, 2, 4])

In [4]:
obj3.reindex(range(6), method = 'ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

`reindex`也可以对 row, columns 进行修改：

In [5]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), index = ['a', 'c', 'd'], columns = ['Ohio', 'Texas', 'California'])

In [6]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [7]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

In [8]:
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [9]:
states = ['Texas', 'Utah', 'California']

In [10]:
frame.reindex(columns = states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [11]:
frame.loc[['a', 'b', 'c', 'd'], states]

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

pandas 现在这样的写法已经不允许了，会返回 Error：`KeyError: Passing list-likes to .loc or [] with any missing labels is no longer supported`；请查看 [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) 以了解`loc`的最新用法 

以下是常用的`reindex`function:

Argument | Description
:- | :-
`index` | 用 sequence 来进行索引改写
`method` | 上面例子说明了一种情况，具体操作是`ffill` -- 向前填充；`bfill` -- 向后填充
`fill_value` | 当 reindexing 时候自动向缺失数据进行补充
`limit` | fill 的时候，设立 maximum size gap（in number of elements）
`tolerance` | fill 的时候，对 inexact matches 设立 maximum size gap（in absolute numeric distance）
`level` | 在 MultiIndex 级别上对 simple index 进行匹配；否则则对子集进行相关操作
`copy` | 如果为 `True`，则即使新索引等于旧索引，也始终复制数据；如果为 `False`，则当索引相等时不要复制数据

### Dropping Entries from an Axis

在 DataFrame 和 object 都可以通过 `drop` 来删除某一条目（行列都没问题）

In [12]:
obj = pd.Series(np.arange(5.), index = ['a', 'b', 'c', 'd', 'e'])

In [13]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [14]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)), index=['Ohio', 'Colorado', 'Utah', 'New York'], columns=['one', 'two', 'three', 'four'])

In [15]:
data.drop('two', axis = 1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [16]:
data.drop(['two', 'four'], axis = 'columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


许多像 `drop` 这一类对 Series 和 DataFrame 形状或大小进行修改的方程，都可以通过 `inplace` 来决定是否返回新的对象：[stackoverflow explained](https://stackoverflow.com/questions/43893457/understanding-inplace-true)

要小心使用 `inplace` 因为任何被删除的数据将会被永久损坏

In [17]:
obj.drop('c', inplace = True)

In [18]:
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

### Indexing, Selection, and Filtering 

Series 的索引和 NumPy 的十分相像，但 pandas 里面可以直接使用索引值（index value）而不是仅仅只能通过 integer 来进行索引；

对于索引值是 labels 的切片（slicing）会与 Python 原生的有一点不同 -- **end point 是包含在内的**

而对于 DataFrame 而言，indexing 一样可以从 single value 或者 sequence 操作；但也有一些 special cases：

1. 用 Boolean 数组来进行切片或者数据选择
2. 用 Boolean DataFrame 来索引，例如 scalar comparison

In [2]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [3]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [4]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [5]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [6]:
data[data < 5] = 0

In [7]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selection with loc and iloc

`loc` （轴标签）和 `iloc` （integer）可以被用于选择 DataFrame 中的行和列以形成新的子集

In [8]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [9]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

以下是常用的 DataFrame Indexing fuctions:

Type | Notes
:- | :-
`df[val]` | 最基本的 indexing
`df.loc[val]` | 通过标签选择行或行组
`df.loc[:, val]` | 通过标签选择列或列组
`df.loc[val1, val2]` | 通过标签同时选择行列
`df.iloc[where]` | 通过 integer 选择行或行组
`df.iloc[:, where]` | 通过 integer 选择列或列组
`df.iloc[where_i, where_j]` | 通过 integer 同时选择行列
`df.at[label_i, label_j]` | 通过行列标签选择单一标量值
`df.iat[i, j]` | 通过行列 integer 选择单一标量值
`reindex` method | 通过标签来选择行或列
`get_value`, `set_value` methods | 通过行列标签来选择单一值

### Integer Indexes 

pandas 对象如果通过 integer 来索引会存在一些问题，比起用非 integer 来索引（不存在任何歧义）。

为了保持一致，如果轴索引包含整数，则数据选择将始终面向标签。 为了更精确地处理，请使用 `loc`（用于标签）或 `iloc`（用于整数）。

### Arithmetic and Data Alignment

pandas 有强大的不同索引的对象之间的算术行为，例如将不同的对象加在一起的时候，如果任何索引对不相同，则结果中的各个索引将是索引对的并集，这跟 database 里面 outer join 的性质是一样的。

内部数据合并的时候，任何有不重叠的标签对应位置将会引入 NaN，而这一缺失值将会在后续的算术中继续传递。

In [10]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
s1
s2
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

对于 DataFrame 而言，这个操作是在行列同时进行的。同样的任何有不重叠的标签将会被替换成 NaN

In [11]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'), index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1
df2
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


如果这个 DataFrame 没有行或列标签的话，返回的结果将会全变成 NaN

In [12]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


#### Arithmetic methods with fill values

为了避免出现 NaN 的情况，我们需要通过`fill_value`来进行算术操作，同样的方法可以用在 reindexing 上

In [13]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
df2.loc[1, 'b'] = np.nan # transform second row second column figure into NaN
df1.add(df2, fill_value = 0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [14]:
df1.reindex(columns = df2.columns, fill_value = 0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


In [15]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


In [16]:
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


以下是常用的算术运行：

Method | Description
:- | :-
`radd` | 加
`rsub` | 减
`rdiv` | 除
`rfloordiv` | 底数除法
`rmul` | 乘
`rpow` | 幂

#### Operations between DataFrame and Series

In [17]:
arr = np.arange(12.).reshape((3, 4))
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

当我们从 arr 中进行减法的时候，这个过程是对于每一行进行一次的，我们称它为 broadcasting。DataFrame 和 Series 的逻辑是一致的：

In [18]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


同样如果 DataFrame 或者 Series 中的索引不重叠，所返回的值会出现 NaN

如果你想对于每一列进行 broadcast 的话，我们需要通过算术方法来进行：

In [19]:
series3 = frame['d']
frame.sub(series3, axis = 'index') # The axis number that you pass is the axis to match on

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


### Function Application and Mapping

NumPy 里面之前提到的 *ufuncs* 同样作用于 pandas（例如 `abs`, `sum`, `mean`），而如果我们想将 function 运用到 DataFrame 上，我们可以使用`apply`:

In [20]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
f = lambda x: x.max() - x.min()

In [21]:
frame.apply(f)

b    3.760418
d    2.945716
e    2.074579
dtype: float64

`apply` 默认会对每一行进行操作，而如果想对每行进行操作的话，我们只需要在`apply`的时候指名就可以了：

In [22]:
frame.apply(f, axis = 'columns')

Utah      4.030701
Ohio      2.859170
Texas     1.017501
Oregon    1.778770
dtype: float64

function 所返回的不一定只能是标量值，它还可以返回多个值的 Series：

In [23]:
def f(x):
    return pd.Series([x.min(), x.max()], index = ['min', 'max'])
frame.apply(f)

,b,d,e
min,-1.453026,-0.741789,-1.723309
max,2.307391,2.203927,0.351269


例如你想改变 *frame* 里面的 floating point，你可以套用 Element-wise Python functions 然后使用`applymap`，而 Series 则是直接`map`：

In [24]:
format = lambda x: '%.2f' % x
frame.applymap(format)

,b,d,e
Utah,2.31,-0.74,-1.72
Ohio,-0.66,2.20,0.35
Texas,-0.71,0.31,-0.57
Oregon,-1.45,0.33,-0.38


### Sorting and Ranking

要根据行列的索引进行词典（字母排序 lexicographically）上的排序，则使用`sort_index`就会直接返回所需要的排序。

DataFrame 里我们可以对任意轴进行排序，只需写入 `sort_index(axis = X)`。

默认情况下，数据排序是按照升序排序的，若要调节成降序，只需写入 `sort_index(ascending = False)`。

根据数值对 Series 进行排序，则使用 `sort_values`；任何缺失值将会默认被排序到最后。

对DataFrame进行排序时，可以将一列或多列中的数据用作排序键；我们需要用到`by`进行相关操作：

In [25]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame.sort_values(by = 'b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [26]:
frame.sort_values(by = ['a', 'b']) # Sort by multiple values

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


pandas 的 `rank` 一开始理解可能会有点困难，不是太直观；DataFrame 是可以接受行列的 rank

默认情况下， `rank` 是通过为每组分配平均等级来将数据打破并排序等级的。具体算法请查看 [w3resources](https://www.w3resource.com/pandas/dataframe/dataframe-rank.php#:~:text=The%20rank()%20function%20is,the%20ranks%20of%20those%20values.&text=Index%20to%20direct%20ranking.) 

In [27]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [28]:
obj.rank(method = 'first') # order in which data is observed

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [29]:
# Assign tie values the maximum rank in the group
# rank can also be in descending order, too
obj.rank(ascending = False, method = 'max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

以下是常用的 rank 的 Method 选择

Method | Description
:- | :-
`average` | 默认平均值
`min` | 对整个数组使用最低等级
`max` | 对整个数组使用最高等级
`first` | 通过数值出现顺序进行排序
`dense` | 跟`min`类似，但数组之间的 rank 始终增加1而不是数组中相同元素的数量

### Axis Indexes with Duplicate Labels 

到目前为止，我们研究的所有示例都具有唯一的轴标签（索引值）。尽管许多 pandas 函数（例如`reindex`）都要求标签唯一，但这不是强制性的。

我们可以通过`is_unique`来进行检查标签是否唯一。

当我们索引有不唯一的标签时会返回一个 Series 而唯一的时候则是一个标量值。

这会导致情况变得十分复杂，所以尽量不要使用重复的标签。无论是对 Series 还是 DataFrame，这些规则都是统一的。